# Task 0

In [1]:
# This script imports required libraries for text processing and connects to MongoDB.

from pymongo import MongoClient
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re


In [2]:
# Accessing the MongoDB server running on localhost on port 27017
client: MongoClient = MongoClient('127.0.0.1',27017)

# Selecting the admin database
db = client["admin"]

# Selecting the "10000 tweets" collection in the admin database
tweets = db["10000 tweets"]

# Deleting tweets that are not in English from the collection
tweets.delete_many({"twitter_lang":{"$ne":"en"}})

# Printing the body of the first 20 tweets in the collection
for t in tweets.find({}).limit(20):
    print(t["body"])


register for convergence to hearchellemelbourne talk about how to manage change through digital transformation 
congratulations suzie walker on both your beautiful little man and your fantastic commitment and hard work you 
join us  the hilton sydney  learn how  make analytics pervasive across yr organisation  
wantirna vic au  am temp c rh pct winds nnw   kmh rain today   mm  hpa amp steady vicweather
so much to see and do when you visit central australia we have our own kind of beaches in the redcentrent the 
fridays from pm free bbq bikini girls amp beverages meat raffles around lunch time seafood raffle from pm 
say hello to this gorgeous gingham dress   perfect for a day of play or a party   originally  now only  
great read from mbs alumna leanda lee mba  on mindfulness attention amp balance via macaudailytimes  mbsnow
rt thasosfamily  rts  do it for sos iheartawards bestfanarmy sosfam 
dormant  by james bryron love   asmsg ian 
why i love my kreg jig  read more  kregjig kreg woo

In [3]:
# Clean the body of each tweet in the collection
for i, t in enumerate(tweets.find()):
     
    # Remove non-alphabetic symbols from the tweet text
    t["body"] = re.sub(r"[^\w\s]+", "", t["body"])
    
    # Remove any digits from the tweet text
    t["body"] = re.sub(r"\d+", "", t["body"])
    
    # Remove any URLs from the tweet text
    t["body"] = re.sub(r"http\S+", "", t["body"])
    
    # Remove any links from the tweet text
    t["body"] = re.sub(r"www\S+", "", t["body"])
    
    # Remove any newline characters from the tweet text
    t["body"] = re.sub(r"\n", " ", t["body"])
    
    # Remove any single quotes from the tweet text, while keeping contractions intact
    t["body"] = re.sub("'", "", t["body"])
    
    # Remove any non-alphabetical characters from the tweet text
    t["body"] = re.sub("[^a-z0-9]"," ", t["body"])
    
    # Remove any multiple spaces or white lines from the tweet text
    t["body"] = re.sub(' +', ' ',t["body"])
    
    # Lowercase the entire tweet text
    final = t["body"].lower()
    
    # Update the tweet body in the collection
    filter = {"_id":t["_id"]}
    updates = {"$set":{"body":final}}
    tweets.update_one(filter,updates)


In [26]:
# Printing the body of the tweets to ensure changes
for t in tweets.find({}).limit(10): 
    print (t["body"])

register for convergence to hearchellemelbourne talk about how to manage change through digital transformation 
congratulations suzie walker on both your beautiful little man and your fantastic commitment and hard work you 
join us the hilton sydney learn how make analytics pervasive across yr organisation 
wantirna vic au am temp c rh pct winds nnw kmh rain today mm hpa amp steady vicweather
so much to see and do when you visit central australia we have our own kind of beaches in the redcentrent the 
fridays from pm free bbq bikini girls amp beverages meat raffles around lunch time seafood raffle from pm 
say hello to this gorgeous gingham dress perfect for a day of play or a party originally now only 
great read from mbs alumna leanda lee mba on mindfulness attention amp balance via macaudailytimes mbsnow
rt thasosfamily rts do it for sos iheartawards bestfanarmy sosfam 
dormant by james bryron love asmsg ian 


# Task 1


In [23]:
# Function to remove stopwords from a list of words
def remove_stopwords(words):
    """Returns a list of words without common or unimportant words"""
    result = []
    for w in words:
        if w not in stopwords.words("english"):
            result.append(w)
    return result 

# Function to stem words in a list of tokens
def stem_words(tokens):
    """Returns a list of stemmed words"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

# Function to remove single-character words from a list of words
def remove_single_char(words):
    """Returns a list of words without single-character words"""
    result = []
    for w in words:
        if len(w) > 1:
            result.append(w)
    return result


# Task 1.1

In [14]:
# Import the Rake module
from rake_nltk import Rake

# Create a Rake object
rake_nltk = Rake()

# Function to find keywords in a given text
def find_keywords_in_text(text):
    """Returns a CSV string of extracted keywords from the input text"""
    rake_nltk.extract_keywords_from_text(text) # Extract keywords using Rake
    keyword_extracted = rake_nltk.get_ranked_phrases() # Get the ranked phrases
    keyword_extracted = remove_single_char(keyword_extracted) # Remove single-character keywords
    ek_token = []
    
    # Tokenize and append the extracted keywords to a list
    for keyword in keyword_extracted:
        ek_token.extend(word_tokenize(keyword))
    
    # Join the list of tokens into a CSV string
    csv_keyword = ",".join(ek_token)
    
    return csv_keyword
 

In [16]:
# Loop through each tweet in the collection
for i, t in enumerate(tweets.find()):
    
    # Extract keywords from the tweet body
    csv_keyword = find_keywords_in_text(t["body"])
    
    # Write the list of keywords to a text file
    with open("Keywords.txt", "a") as f:
        f.write(csv_keyword + '\n')
    
    # Update the "Keywords" field in the tweet document in the database
    filter = {"_id":t["_id"]}
    updates = {"$set":{"Keywords":csv_keyword}}
    tweets.update_one(filter,updates)


    

In [18]:
# Printing to the console
for t in tweets.find().limit(20):
    print(t['Keywords'])

manage,change,hearchellemelbourne,talk,digital,transformation,register,convergence
congratulations,suzie,walker,beautiful,little,man,hard,work,fantastic,commitment
make,analytics,pervasive,across,yr,organisation,hilton,sydney,learn,join,us
temp,c,rh,pct,winds,nnw,kmh,rain,today,mm,hpa,amp,steady,vicweather,wantirna,vic,au
visit,central,australia,see,redcentrent,much,kind,beaches
pm,free,bbq,bikini,girls,amp,beverages,meat,raffles,around,lunch,time,seafood,raffle,pm,fridays
gorgeous,gingham,dress,perfect,say,hello,party,originally,play,day
mindfulness,attention,amp,balance,via,macaudailytimes,mbsnow,mbs,alumna,leanda,lee,mba,great,read
sos,iheartawards,bestfanarmy,sosfam,rt,thasosfamily,rts
james,bryron,love,asmsg,ian,dormant
kregjig,kreg,woodworking,pocketholejig,carbatec,kreg,jig,read,love
following,takes,place,pm
killer,us,military,robots,yet,rt,breitbarttech,plans
profits,coming,forex,knowledge,dont,need,need,watch,trade
last,dragon,th,anniversary,bluray,trailer
past,years,bubba,vs,

# Task 1.2

In [80]:
# Function to find named entities in a given text
def find_named_entities_in_text(text):
    """Returns a CSV string of named entities extracted from the input text"""
    
    # Tokenize the input text
    words = word_tokenize(text)
    
    # Remove stopwords from the text
    text = remove_stopwords(words)
    
#     # Stem the words in the text
#     text = stem_words(text)
    
    # Run POS tagging to identify grammatical groupings of keywords
    tagged_words = nltk.pos_tag(text)
    
    # Extract only the tags that match NN, NNS, NNP, or NNPS
    pos_tags = [(word) for word, tag in tagged_words if tag in ["NN", "NNS", "NNP", "NNPS"]]
    
    # Join the list of named entities into a CSV string
    csv_ne = ",".join(pos_tags)
    
    return csv_ne

    

In [81]:
# Loop through each tweet in the collection
for i, t in enumerate(tweets.find()):
    
    # Extract named entities from the tweet body
    csv_entities = find_named_entities_in_text(t["body"])
        
    # Write the list of named entities to a text file
    with open("Named_Entities.txt", "a") as f:
        f.write(csv_entities + '\n')
    
    # Update the "Entities" field in the tweet document in the database
    filter = {"_id":t["_id"]}
    updates = {"$set":{"Entities":csv_entities}}
    tweets.update_one(filter,updates)


In [82]:
# Printing the sample output to console
for tweet in tweets.find().limit(20):
    print(tweet['Entities'])

register,convergence,talk,manage,change,transformation
congratulations,walker,man,commitment,work
sydney,learn,analytics,yr,organisation
wantirna,vic,au,temp,c,rh,pct,kmh,rain,today,hpa,amp,vicweather
australia,kind,beaches,redcentrent
fridays,bbq,bikini,girls,beverages,raffles,lunch,time,seafood,raffle,pm
hello,gingham,dress,day,party
mbs,lee,mba,mindfulness,attention,balance,macaudailytimes
rt,sos,iheartawards,sosfam
james,asmsg
kreg,jig,kreg,pocketholejig,carbatec
place,pm
rt,breitbarttech,plans,robots
dont,forex,trade,need,profits
dragon,bluray,trailer
rt,espngolf,themasters,week,vote,masters,years,vs,spieth
movie
album,synths,groove,add,nuance,fluidity,guitar,force
attack,superrugby,rd,preview,turnovers,brumbiesrugby,bruvchi
article,benefits,mediation


# Task 1.3

In [36]:
# Import required libraries
from gensim import corpora, models
from gensim.utils import simple_preprocess 
from nltk.corpus import stopwords

# Function to find topic in a given text using LDA
def find_Topic_in_text(text):
    """Returns a CSV string of the topic extracted from the input text using LDA"""
    
    # Initialize an empty list to store topic words
    topics_tweet = []
    
    # Initialize an empty string to store the CSV-formatted topic words
    csv_topics = ""
    
    # Tokenize the input text and remove stop words
    words = [word for word in simple_preprocess(text) if word not in stopwords.words("english")]
    
    # Create a dictionary and corpus from the tokenized text
    dictionary = corpora.Dictionary([words])
    corpus = [dictionary.doc2bow(doc) for doc in [words]]
    
    if len(words) != 0:
        # Build an LDA model from the corpus
        lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=1)
   
        # Get the top words for the first topic
        topics = lda_model.get_topic_terms(topicid=0, topn=1)
    
        # Append each word to the topics_tweet list
        for word_id,prob in topics: 
            topics_tweet.append(dictionary[word_id])
    
    if len(topics_tweet) != 0:
        # Join the list of topic words into a CSV string
        csv_topics = ",".join(topics_tweet)
    
    return csv_topics



In [37]:
# Loop through each tweet in the collection
for i, t in enumerate(tweets.find()):
    
    # Extract a topic from the tweet body
    csv_topics = find_Topic_in_text(t["body"])
        
    # Write the topic to a text file
    with open("Topics.txt", "a") as f:
        f.write(csv_topics + '\n')
    
    # Update the "Topics" field in the tweet document in the database
    filter = {"_id":t["_id"]}
    updates = {"$set":{"Topics":csv_topics}}
    tweets.update_one(filter,updates)

    

In [39]:
# Printing sample to the console
for tweet in tweets.find().limit(10):
    print(tweet['Topics'])


change
walker
sydney
rain
kind
pm
dress
alumna
bestfanarmy
ian


# Task 1.4

In [40]:
# Import required library
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Function to find sentiment in a given text using VADER
def find_sentiment_in_text(text):
    """Returns a CSV string of the sentiment extracted from the input text using VADER"""
    
    # Initialize an empty list to store sentiments
    sentiments = []
    
    # Initialize an empty string to store the CSV-formatted sentiment
    sentiments_csv = ""
    
    # Create a SentimentIntensityAnalyzer object
    sid = SentimentIntensityAnalyzer()

    # Analyze the sentiment of the input text
    scores = sid.polarity_scores(text)

    # Select the sentiment with the highest score
    sentiment = max(scores, key=scores.get)

    # Append the sentiment with the highest score to the sentiments list
    sentiments.append(sentiment)
    
    # Join the list of sentiments into a CSV string
    csv_sentiments = ",".join(sentiments)
    
    return csv_sentiments


In [43]:
# Loop through each tweet in the collection
for i, t in enumerate(tweets.find()):
    
    # Extract a sentiment from the tweet body
    csv_sentiments = find_sentiment_in_text(t["body"])
    
    # Write the sentiment to a text file
    with open("Sentiments.txt", "a") as f:
        f.write(csv_sentiments + '\n')
    
    # Update the "Sentiments" field in the tweet document in the database
    filter = {"_id":t["_id"]}
    updates = {"$set":{"Sentiments":csv_sentiments}}
    tweets.update_one(filter,updates)

    

In [44]:
#Printing the sample to console
for tweet in tweets.find().limit(20):
    print(tweet['Sentiments'])


neu
compound
neu
neu
neu
neu
compound
neu
neu
compound
neu
neu
neu
neu
neu
neu
neu
neu
neu
compound


# Task 2

In [45]:
# Import required libraries
import pymongo
from nltk.tokenize import word_tokenize

# Extract tweet text and clean/tokenize
tweet_text = []
for t in tweets.find({}):
    text = t["body"]
    text = word_tokenize(text)
    text = ",".join(text)
    tweet_text.append(text)

# Write tweet text to file
with open("tweet_text.txt", "w") as f:
    for text in tweet_text:
        if text:
            f.write(text + "\n")



In [47]:
# Running the CountWordOccurrence Class defined in a seprate PY notebook

import subprocess
import os

input_file = os.path.join(os.getcwd(), 'tweet_text.txt')
output_file = os.path.join(os.getcwd(), 'MRCountOccurrence.txt')

result = subprocess.run(['python', 'MRCountOccurrence.py', input_file], capture_output=True)

with open(output_file, 'w') as f:
    f.write(result.stdout.decode())


In [48]:
# Printing the sample output in the console

with open('MRCountOccurrence.txt', 'r') as f:
    lines = f.readlines()



for line in lines:
    word, count = line.strip().split('\t')
    print(f'{word:<20}{count}')

"pips"              14
"pipstealer"        2
"pirate"            2
"piratelifebeer"    1
"pisa"              1
"piss"              4
"pissed"            3
"pisser"            1
"pisses"            1
"pistol"            1
"pistols"           1
"pitch"             6
"pitches"           4
"pitchforderin"     1
"pitching"          2
"pither"            1
"pitt"              2
"pittatheist"       1
"pitted"            1
"pix"               1
"pixel"             2
"piyush"            1
"pizerria"          1
"pizza"             3
"pizzazz"           1
"pjhelliar"         1
"pjk"               1
"pjnet"             2
"pjs"               1
"pk"                2
"pkm"               1
"pl"                2
"pla"               1
"plabg"             1
"place"             26
"placed"            2
"placement"         1
"placeofworkout"    1
"places"            4
"placing"           1
"plain"             1
"plainly"           2
"plan"              47
"plane"             6
"planenut"          1
"planes

"rice"              4
"rich"              6
"richard"           2
"richardajkeys"     1
"richardbranson"    2
"richarddinatale"   2
"richardlidstone"   1
"richardmclellan"   1
"richardridge"      1
"richardson"        2
"richest"           1
"richforrest"       2
"richie"            3
"richmond"          7
"richmondvfl"       1
"richvos"           1
"rick"              6
"rickettsa"         1
"rickrwells"        1
"rickshaw"          1
"ricky"             1
"rickyhil"          1
"rickypdillon"      1
"rid"               4
"ride"              9
"rider"             2
"riderfitcc"        1
"riders"            2
"rides"             2
"ridesharing"       2
"ridiculous"        6
"riding"            3
"rielresearch"      1
"rieter"            1
"rieva"             1
"riff"              1
"rifflesby"         1
"riffs"             1
"rifle"             1
"rift"              1
"rigged"            1
"right"             158
"rightcs"           1
"righteous"         1
"rightfully"        1
"rightgi

"slot"              2
"slow"              8
"slowed"            1
"slowest"           1
"slowing"           1
"slowly"            10
"slows"             1
"slowtravel"        1
"slug"              2
"slugger"           1
"slumps"            1
"slut"              1
"slutcouture"       3
"slutty"            1
"smackdown"         9
"smacked"           1
"smail"             1
"small"             21
"smallball"         1
"smallbiz"          1
"smallbiztrump"     1
"smallbusiness"     1
"smalldogs"         1
"smaller"           2
"smallest"          1
"smalltown"         1
"smariv"            1
"smart"             14
"smartbribe"        1
"smartcompany"      1
"smarter"           3
"smartwatches"      1
"smash"             3
"smashcon"          2
"smashed"           2
"smashprojjff"      1
"smashwords"        1
"smb"               1
"smc"               1
"smell"             1
"smelt"             1
"smes"              1
"smfc"              2
"smh"               26
"smharmitage"       2
"smiel

"grabgate"          11
"grabs"             3
"grace"             5
"gracefulcurls"     1
"gracemurphy"       1
"gracetown"         1
"graciemaebeagle"   1
"grade"             3
"gradebook"         1
"gradon"            1
"graduatejob"       1
"graduates"         3
"graeme"            1
"graemesamuel"      1
"graffiti"          1
"graham"            1
"grains"            1
"grammatically"     1
"grammy"            1
"grams"             1
"grand"             10
"grande"            1
"grandfather"       2
"grandkids"         1
"grandma"           1
"grandpa"           1
"grandparents"      1
"grandpas"          1
"grandpayoongii"    1
"grandtheft"        1
"grange"            1
"grangegolf"        1
"grangers"          1
"granite"           1
"granny"            1
"grant"             4
"granted"           3
"grants"            4
"grapefruit"        1
"graphene"          1
"graphic"           4
"graphics"          1
"grass"             2
"grassfed"          1
"grassfire"         1
"grassro

"boxes"             4
"boxing"            2
"boy"               12
"boycameron"        1
"boycey"            1
"boycott"           3
"boycottwnba"       1
"boyd"              1
"boyfriend"         3
"boyle"             1
"boys"              20
"boysnoize"         1
"boyswithearts"     4
"boyz"              1
"bp"                3
"bpi"               1
"bpreiss"           1
"br"                1
"bra"               2
"braaaandonn"       1
"brace"             1
"bracelet"          1
"bracket"           1
"brad"              4
"bradenjohn"        1
"bradford"          1
"bradjgray"         2
"bradley"           1
"bradon"            1
"brag"              1
"brahhhhh"          1
"braies"            1
"brain"             13
"brained"           1
"brainpicker"       1
"brainpower"        1
"brains"            1
"brake"             2
"brakes"            1
"brampton"          1
"branch"            1
"branches"          1
"brand"             12
"brandawareness"    1
"branded"           1
"brand

...

"appointment"       1
"appointments"      1
"appoints"          1
"appreciate"        8
"appreciated"       5
"appreciation"      5
"apprentices"       1
"approach"          5
"approaching"       1
"approval"          1
"approve"           2
"approved"          1
"approximately"     1
"apps"              5
"appsforgood"       1
"appstoreoz"        1
"apr"               5
"apri"              1
"april"             235
"aprilfools"        36
"aprilfoolsday"     16
"aprilfoolsisannoying"1
"aprils"            4
"apriyl"            1
"apryl"             3
"apxwest"           1
"aqiskincare"       1
"aquamarine"        1
"aquarium"          1
"aquatic"           1
"aquaticdelorean"   1
"aquifers"          1
"ar"                2
"ara"               1
"arab"              2
"arabia"            3
"arabic"            1
"araieij"           1
"arasosar"          1
"arbvter"           1
"arc"               2
"arcacia"           1
"arcade"            2
"arcadesat"         1
"arcadiagt"         1
"arc

"ifwtwa"            1
"ifyoulovequotes"   1
"ig"                3
"igbo"              2
"igetapackagethatsawesome"1
"iggy"              2
"iggyazalea"        1
"iggys"             1
"ight"              1
"ign"               2
"ignites"           1
"ignorance"         2
"ignorant"          1
"ignore"            5
"ignored"           2
"ignoring"          5
"igrigorik"         1
"igsouthwestsaltwater"4
"igstephtee"        1
"ihabx"             1
"iharry"            1
"ihe"               1
"iheart"            2
"iheartaw"          1
"iheartawards"      249
"iheartradio"       31
"ihearttheaterny"   1
"ihjc"              1
"iho"               1
"ihopkc"            2
"ihr"               1
"ii"                1
"iiamit"            1
"iiesppltell"       1
"iii"               3
"iinc"              1
"iinet"             3
"iirc"              1
"iisam"             1
"iisuperwomanii"    1
"ikea"              1
"ikeas"             1
"ikickeducks"       1
"ikr"               2
"il"                3

"kop"               1
"kordei"            1
"korea"             10
"koreaboo"          1
"korean"            5
"koreykuhl"         1
"korinspiration"    1
"kph"               2
"kpimethod"         1
"kplyley"           2
"kpmgaustralia"     1
"kpmgs"             1
"kpop"              2
"kpopgainmutuals"   2
"kpopperfect"       1
"kr"                1
"krakkerznfl"       1
"krazykid"          2
"kreg"              2
"kregjig"           1
"kreme"             2
"krispy"            2
"krissiamiet"       1
"kristen"           1
"kristenhazard"     1
"kristenobv"        1
"kristian"          1
"kristicolleen"     1
"kristinaantuna"    1
"kristy"            1
"kronik"            2
"krooked"           1
"krum"              1
"krunalumb"         1
"kruse"             2
"krystal"           1
"krystals"          2
"ks"                1
"kschwantzs"        1
"kseibertau"        2
"ksltv"             1
"ktmbikesaus"       1
"ktnepal"           1
"kts"               5
"kudos"             2
"kumardee

"airgtgtgtgt"       1
"airin"             1
"airing"            2
"airlift"           2
"airlineguys"       1
"airlines"          1
"airmail"           1
"airplane"          1
"airpollution"      1
"airport"           13
"airportline"       1
"airportmany"       1
"airtasker"         20
"airtime"           1
"airtool"           1
"airwaves"          1
"airways"           3
"aishprobs"         1
"aisle"             1
"aj"                3
"ajenglish"         1
"ajfsolidarity"     1
"ajinkyarahane"     1
"ajplus"            2
"ajstylesorg"       1
"ak"                1
"aka"               5
"akalagianblunt"    1
"akard"             1
"akasike"           1
"akbag"             1
"akhinthobe"        1
"aki"               1
"akidearest"        1
"akira"             1
"akon"              1
"akqld"             2
"al"                21
"ala"               6
"alabama"           3
"alabaster"         1
"alam"              1
"alan"              2
"alana"             2
"alanalda"          1
"alanam

"orderr"            1
"orders"            3
"ordinary"          1
"ore"               2
"oreilly"           1
"org"               2
"organa"            1
"organisation"      2
"organisational"    1
"organise"          1
"organised"         1
"organizer"         1
"organo"            1
"organogold"        1
"orgs"              1
"ori"               1
"oricon"            1
"orig"              1
"origin"            5
"origina"           1
"original"          6
"originalfunko"     2
"originally"        1
"originals"         1
"origindatajam"     1
"orjp"              1
"orlando"           2
"orleans"           1
"ornamental"        1
"ornl"              1
"orphan"            1
"orphanage"         1
"orphanblack"       3
"orpheusbleeds"     1
"orringa"           1
"os"                3
"osama"             1
"osborne"           1
"oscar"             1
"oscars"            1
"oski"              1
"oskiandfig"        1
"osws"              1
"ot"                1
"otakebi"           1
"otaku"   

# Task 3

In [49]:
# Extract the IDs and localities of Australian tweets
australian_tweets = [(tweet['id'], tweet['gnip']['profileLocations'][0]['address'].get('locality', 'Locality not mentioned'))
                    for tweet in tweets.find()
                    if 'gnip' in tweet
                    and 'profileLocations' in tweet['gnip']
                    and 'address' in tweet['gnip']['profileLocations'][0]
                    and 'country' in tweet['gnip']['profileLocations'][0]['address']
                    and tweet['gnip']['profileLocations'][0]['address']['country'] == 'Australia']

# Write the IDs and localities to a text file
with open('australian_tweets.txt', 'w') as file:
    file.writelines([f"{tweet_id}, {locality}\n" for tweet_id, locality in australian_tweets])


In [52]:
# Running the CountTweetCity Class defined in a seprate PY notebook

import subprocess
import os

input_file = os.path.join(os.getcwd(), 'australian_tweets.txt')
output_file = os.path.join(os.getcwd(), 'CountTweetCity.txt')

result = subprocess.run(['python', 'Count_City.py', input_file], capture_output=True)

with open(output_file, 'w') as f:
    f.write(result.stdout.decode())

In [53]:
# Printing the sample output to console


# Read the lines from the australian_tweets.txt file
with open('CountTweetCity.txt', 'r') as f:
    lines = f.readlines()

# Print the lines
for line in lines:
    city, count = line.strip().split('\t')
    print(f'{city:<20}{count}')

" Tamworth"         1
" Toodyay"          2
" Toowoomba"        3
" Townsville"       1
" Traralgon"        1
" Victoria Park"    12
" West Melbourne"   1
" West Perth"       2
" Whyalla"          1
" Wodonga"          2
" Wollongong"       7
" Yeppoon"          1
" York"             1
" Mackay"           3
" Mandurah"         2
" Melbourne"        844
" Melton"           1
" Menai"            1
" Moe"              1
" Mooloolaba"       1
" Mount Lawley"     1
" Newcastle"        17
" Noosa Heads"      1
" North Booval"     2
" North Ipswich"    1
" North Sydney"     2
" Parap"            1
" Penrith"          1
" Perth"            312
" Phillip Island"   1
" Port Douglas"     2
" Port Macquarie"   1
" Port Melbourne"   1
" Portland"         1
" Redfern"          2
" Richmond"         4
" Roleystone"       1
" Sawyers Valley"   1
" South Brisbane"   3
" South Melbourne"  445
" South Perth"      1
" Southbank"        1
" Sydney"           1446
" Broome"           1
" Brunswick"        1

# Task 4

In [70]:
# Use set comprehension to extract unique tweet IDs
tweet_ids = {tweet['id'].split(':')[-1] for tweet in tweets.find()}

# Write the tweet IDs to a file using with statement and join method
with open('tweet_id.txt', 'w') as f:
    f.write('\n'.join(tweet_ids))


In [77]:
# Running the MapReduceMergeSort Class defined in a seprate PY notebook

import subprocess
import os

input_file = os.path.join(os.getcwd(), 'tweet_id.txt')
output_file = os.path.join(os.getcwd(), 'tweets_SortedID.txt')

result = subprocess.run(['python', 'MapReduceMergeSort.py', input_file], capture_output=True)

with open(output_file, 'w') as f:
    f.write(result.stdout.decode())

# Task 4 Part 2

In [74]:
# Mergesort function without map reduce

def merge(left, right):
    result = []
    i, j = 0, 0
    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result += left[i:]
    result += right[j:]
    return result

def merge_sort(ids):
    if len(ids) <= 1:
        return ids
    mid = len(ids) // 2
    left = ids[:mid]
    right = ids[mid:]
    left = merge_sort(left)
    right = merge_sort(right)
    return merge(left, right)

def sort_ids(filename, output_filename):
    with open(filename, 'r') as f:
        ids = f.read().split()
    ids = merge_sort(ids)
    with open(output_filename, 'w') as f:
        for id in ids:
            f.write(id + '\n')



In [76]:
# Using the Code defined above to on id list to obtain sorted id
sort_ids('tweet_id.txt', 'MergeSortID.txt')



# Task 5

In [90]:
import dask.bag as db
import dask.dataframe as dd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from dask.dataframe import from_pandas
import pandas as pd

# Extracting keywords from tweets
all_keywords = []
for tweet in tweets.find():
    keywords = tweet['Keywords'].split(',')
    all_keywords.extend(keywords)
    
# Converting list of keywords to Dask Bag
keywords_bag = db.from_sequence(all_keywords)

# Converting Bag to DataFrame
df_keywords = db.Bag.to_dataframe(keywords_bag, columns=['Keywords'])

# Counting keyword occurrences and creating a new DataFrame
df_counts = df_keywords.groupby('Keywords').Keywords.count().to_frame('count').reset_index()

# Extracting unique keywords
unique_keywords = list(df_counts['Keywords'].unique())

# Creating a Dask Bag of tweet texts and counting the total number of documents
texts = db.from_sequence([tweet['body'] for tweet in tweets.find()])
total_documents = texts.count().compute()

# Tokenizing the texts into lists of words
texts_tokenized = texts.map(lambda text: TextBlob(text).words)

# Flattening the lists of words into a single Dask Bag of words
texts_flattened = texts_tokenized.flatten()

# Initializing a dictionary to store the word counts for each unique keyword
word_counts = {keyword: 0 for keyword in unique_keywords}

# Counting the number of occurrences of each word in the flattened texts
for word in texts_flattened:
    if word in word_counts:
        word_counts[word] += 1

# Calculating the total number of words in the texts
total_words = texts_flattened.count().compute()

# Initializing a dictionary to store the document count for each unique keyword
doc_counts = {keyword: 0 for keyword in unique_keywords}

# Counting the number of documents in which each keyword appears
for sentence in texts_tokenized:
    for word in set(sentence):
        if word in doc_counts:
            doc_counts[word] += 1

# Creating a Pandas DataFrame with counts and frequencies
df = dd.from_pandas(pd.DataFrame({
    'Keywords': unique_keywords,
    'count': [word_counts[keyword] for keyword in unique_keywords],
    'term_freq': [word_counts[keyword] / total_words for keyword in unique_keywords],
    'doc_freq': [doc_counts[keyword] for keyword in unique_keywords],
}), npartitions=3)

# Calculating IDF and TF-IDF for each keyword and adding as columns to the DataFrame
df['idf'] = np.log((1 + total_documents) / (1 + df['doc_freq']))
df['tf-idf'] = df['term_freq'] * df['idf']

# Sorting the Dask DataFrame by TF-IDF score in descending order
ddf_sorted = df.sort_values(by=['tf-idf'], ascending=False)

# Selecting the top N keywords with highest TF-IDF scores
N = 20
top_keywords = ddf_sorted.head(N)

# Printing the top N keywords and their corresponding TF-IDF scores
print(top_keywords)



          Keywords  count  term_freq  doc_freq       idf    tf-idf
489             rt   3858   0.031442      3778  0.850280  0.026735
3839           not    497   0.004050       464  2.945457  0.011930
7640            me    440   0.003586       414  3.059216  0.010970
262             im    404   0.003293       368  3.176698  0.010459
15             amp    402   0.003276       370  3.171292  0.010390
379            new    343   0.002795       320  3.316053  0.009270
3         activist    314   0.002559       313  3.338102  0.008542
318           like    304   0.002478       284  3.435005  0.008510
135            day    287   0.002339       274  3.470723  0.008118
579         threat    289   0.002355       283  3.438520  0.008099
3686           can    274   0.002233       264  3.507765  0.007833
395            one    263   0.002143       251  3.558065  0.007626
1164         union    260   0.002119       250  3.562042  0.007548
872            get    252   0.002054       242  3.594433  0.00